In [2]:
import findspark


In [3]:
from pyspark.sql import SparkSession

In [4]:
from pyspark.conf import SparkConf
from delta import configure_spark_with_delta_pip
conf = SparkConf()
conf.set("spark.driver.maxResultSize", "8g")
conf.set("spark.driver.memory", "8g") 
conf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
conf.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")

builder = SparkSession.builder.appName("dev_test_notebook_2") \
    .enableHiveSupport()\
    .config(conf=conf)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jupyter/.ivy2/cache
The jars for the packages stored in: /home/jupyter/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7565b65f-ff5f-403a-beca-fcb67252f725;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-core_2.12/2.4.0/delta-core_2.12-2.4.0.jar ...
	[SUCCESSFUL ] io.delta#delta-core_2.12;2.4.0!delta-core_2.12.jar (1640ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/2.4.0/delta-storage-2.4.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;2.4.0!delta-storage.jar (1183ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.9.3/antlr4-runtime-4.9.3.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.9.3!antlr4-runtime.jar (180ms)
:: resolution report :: resolve 14550ms :: 

In [5]:
spark.sql("CREATE TABLE IF NOT EXISTS src_test2 (key INT, value STRING) USING hive")
spark.sql("LOAD DATA LOCAL INPATH '/opt/spark/examples/src/main/resources/kv1.txt' INTO TABLE src_test2")

00:17:04.832 [Thread-5] ERROR org.apache.hadoop.hive.common.FileUtils - The jar file path /opt/hive/lib/delta-hive-assembly_2.12-0.6.0.jar doesn't exist
00:17:05.290 [Thread-5] ERROR org.apache.hadoop.hive.common.FileUtils - The jar file path /opt/hive/lib/delta-hive-assembly_2.12-0.6.0.jar doesn't exist
00:17:06.242 [Thread-5] ERROR org.apache.hadoop.hive.common.FileUtils - The jar file path /opt/hive/lib/delta-hive-assembly_2.12-0.6.0.jar doesn't exist
00:17:07.936 [Thread-5] ERROR org.apache.hadoop.hive.common.FileUtils - The jar file path /opt/hive/lib/delta-hive-assembly_2.12-0.6.0.jar doesn't exist


DataFrame[]

In [6]:
spark.sql("SELECT * FROM src_test2").show()

+---+-------+
|key|  value|
+---+-------+
|238|val_238|
| 86| val_86|
|311|val_311|
| 27| val_27|
|165|val_165|
|409|val_409|
|255|val_255|
|278|val_278|
| 98| val_98|
|484|val_484|
|265|val_265|
|193|val_193|
|401|val_401|
|150|val_150|
|273|val_273|
|224|val_224|
|369|val_369|
| 66| val_66|
|128|val_128|
|213|val_213|
+---+-------+
only showing top 20 rows



In [7]:
spark.sql("SELECT COUNT(*) FROM src_test2").show()

+--------+
|count(1)|
+--------+
|     500|
+--------+



In [8]:
sqlDF = spark.sql("SELECT key, value FROM src_test2 WHERE key < 10 ORDER BY key")

In [9]:
stringsDS = sqlDF.rdd.map(lambda row: "Key: %d, Value: %s" % (row.key, row.value))
for record in stringsDS.collect():
    print(record)

Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 2, Value: val_2
Key: 4, Value: val_4
Key: 5, Value: val_5
Key: 5, Value: val_5
Key: 5, Value: val_5
Key: 8, Value: val_8
Key: 9, Value: val_9


In [10]:
from pyspark.sql import Row

Record = Row("key", "value")
recordsDF = spark.createDataFrame([Record(i, "val_" + str(i)) for i in range(1, 101)])
recordsDF.createOrReplaceTempView("records")

In [11]:
spark.sql("SELECT * FROM records r JOIN src_test2 s ON r.key = s.key").show()

+---+------+---+------+
|key| value|key| value|
+---+------+---+------+
|  2| val_2|  2| val_2|
|  4| val_4|  4| val_4|
|  5| val_5|  5| val_5|
|  5| val_5|  5| val_5|
|  5| val_5|  5| val_5|
|  8| val_8|  8| val_8|
|  9| val_9|  9| val_9|
| 10|val_10| 10|val_10|
| 11|val_11| 11|val_11|
| 12|val_12| 12|val_12|
| 12|val_12| 12|val_12|
| 15|val_15| 15|val_15|
| 15|val_15| 15|val_15|
| 17|val_17| 17|val_17|
| 18|val_18| 18|val_18|
| 18|val_18| 18|val_18|
| 19|val_19| 19|val_19|
| 20|val_20| 20|val_20|
| 24|val_24| 24|val_24|
| 24|val_24| 24|val_24|
+---+------+---+------+
only showing top 20 rows



In [12]:
spark.sql("SELECT * FROM default.cr_table_demo_delta").show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `default`.`cr_table_demo_delta` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [default, cr_table_demo_delta], [], false


In [ ]:
spark.stop()